# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
ont_file_name = "ontology.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

# Nom du répertoire GraphDB
repository_name = "addresses_from_factoids"

# Définition des graphes nommés
ontology_named_graph_name = "ontology"
facts_graph_name = "facts"
factoids_graph_name = "factoids"

# URIs pour accéder aux logiciels GraphDB et Ontotext-Refine
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
gd.remove_repository(graphdb_url, repository_name)
gd.create_config_local_repository_file(local_config_file, repository_name, ruleset_file=ruleset_file, disable_same_as=False)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `repository_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, repository_name)

## Import de l'ontologie

In [ ]:
gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, ont_file, ontology_named_graph_name)

## Définition de variables liées aux sources

### Voies de Paris via Wikidata

`wdpt` pour "wikidata paris thoroughfares"


In [ ]:
# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(tmp_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_voies_wikidata.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

`vpt` pour "ville paris thoroughfares"

In [ ]:
# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

`bpa` pour "BAN paris addresses"

In [ ]:
# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

## Fonctions pour des traitements de construction des graphes de factoïdes

### Ajout de labels normalisés

* Étant donné les variations de labels qui sont similaires, la normalisation consiste ici à :
    * la mise en minuscule des caractères
    * suppression des signes diacritiques (accents, cédille...)
    * suppression de termes "inutiles" : déterminants, prépositions...
* Ces labels normalisés sont liés aux repères via `skos:hiddenLabel`

In [ ]:
def add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri:URIRef):
    """
    Ajout de labels normalisés pour les repères via `skos:hiddenLabel` dans le graphe nommé temporaire `tmp_graph_uri`
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; addr:hasAttribute [a addr:Attribute; addr:isAttributeType atype:NameAttribute; addr:version [a addr:AttributeVersion; addr:versionValue {label_var}]].
            BIND({norm_label_function} AS {norm_label_var})
        }}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

### Suppression des instants temporels sans timeStamp

Il existe des resources de classe `addr:TimeInstant` qui ont un calendrier et une granularité dans avoir de time stamp. Il faut supprimer ces ressources qui n'ont aucune utilité.

In [ ]:
def remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri:URIRef):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    DELETE {{
        ?timeInstant ?p ?o.
        ?s ?p ?timeInstant.
    }}WHERE {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?timeInstant a addr:TimeInstant.
            MINUS {{?timeInstant addr:timeStamp ?timeStamp}}
            {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
        }}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

### Ajout d'un lien entre les repères et les sources

Un factoïde est la représentation d'une information, d'un fait dans une source. Ici, on crée des repères qui ont une identité définie par la source. Ces derniers doivent avoir un lien avec la source afin d'attester la provenance de son existence.

Pour cela, on sélectionne l'ensemble des repères (`?landmark`) situés dans le graphe nommé défini par `factoids_graph_uri` afin de créer le triplet `<?landmark rico:isOrWasDescribedBy ?sourceUri>` où `?sourceUri` est l'URI décrivant la source.

In [ ]:
def add_factoids_resources_links(graphdb_url, repository_name, factoids_refactoids_uri:URIRef, factoids_graph_uri:URIRef):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

    INSERT {{
        GRAPH ?g {{
            ?elem rico:isOrWasDescribedBy {factoids_refactoids_uri.n3()}.
        }}
    }}WHERE {{
        {{?type rdfs:subClassOf* addr:Landmark}} UNION
        {{?type rdfs:subClassOf* addr:LandmarkRelation}} UNION
        {{?type rdfs:subClassOf* addr:AttributeVersion}} UNION
        {{?type rdfs:subClassOf* addr:Address}} UNION
        {{?type rdfs:subClassOf* addr:Change}} UNION
        {{?type rdfs:subClassOf* addr:Event}} UNION
        {{?type rdfs:subClassOf* addr:TemporalEntity}}

        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?elem a ?type.
        }}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

### Conversion du fichier brut vers le graphe dans GraphDB

À partir un fichier brut (un fichier tabulaire comme un CSV), la fonction le convertit en graphe de connaissances dans un fichier `ttl` (ici `kg_file`). La manière de convertir le fichier est défini par le fichier `ontorefine_mapping_file`, la conversion se fait par Ontotext Refine. Par la suite, le fichier `ttl` est importé dans le répertoire dont le nom est `repository_name`, plus précisément dans le graphe nommé `graph_name`.

In [ ]:
def from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, graph_name, csv_file, ontorefine_mapping_file, kg_file):
    # Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
    otr.get_export_file_from_ontorefine(csv_file, ontorefine_mapping_file, kg_file, ontorefine_cmd, ontorefine_url, repository_name)

    # Importer le fichier `kg_file` qui a été créé lors de la ligne précédente dans le répertoire `repository_name`, dans le graphe nommé `graph_name` 
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, kg_file, graph_name)

## Construction des factoïdes pour chaque source

### Nomenclature des voies de la ville de Paris

#### Création des données liées à la source

In [ ]:
def create_factoids_ville_paris(graphdb_url, repository_name, factoids_refactoids_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {factoids_refactoids_uri.n3()} a rico:Record;
                    rdfs:label "dénomination des voies de Paris (actuelles et caduques)"@fr;
                    rico:hasPublisher facts:DirTopoDocFoncVP .
                facts:DirTopoDocFoncVP a rico:CorporateBody;
                    rdfs:label "Département de la Topographie et de la Documentation Foncière de la Ville de Paris"@fr.    
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Définition d'un processus de création des données de la Ville de Paris

In [ ]:
def create_factoid_process_ville_paris(graphdb_url, repository_name,
                                       ontorefine_url, ontorefine_cmd,
                                        factoids_graph_name, facts_graph_name, 
                                        vpta_csv_file, vptc_csv_file,
                                        vpta_mapping_file, vptc_mapping_file,
                                        vpta_kg_file, vptc_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la dénomination des voies de la ville de Paris
    """

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))

    # A partir des fichiers csv décrivant les voies de la ville de Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, vpta_csv_file, vpta_mapping_file, vpta_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, vptc_csv_file, vptc_mapping_file, vptc_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)
    
    # L'URI ci-dessous définit la source liée à la ville de Paris
    vdp_factoids_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_VDP")
    create_factoids_ville_paris(graphdb_url, repository_name, vdp_factoids_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)

    # Ajout de liens entre les ressources de type repère et la source
    add_factoids_resources_links(graphdb_url, repository_name, vdp_factoids_uri, factoids_graph_uri)

### Base Adresse Nationale (BAN)

#### Création des données liées à la source

In [ ]:
def create_factoids_ban(graphdb_url, repository_name, factoids_refactoids_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {factoids_refactoids_uri.n3()} a rico:Record;
                    rdfs:label "Base Nationale Adresse"@fr;
                    rico:hasPublisher facts:DINUM_ANCT_IGN .
                facts:IGN_DINUM_ANCT a rico:CorporateBody;
                    rdfs:label "DINUM / ANCT / IGN"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Nettoyage du graphe

Détection de repères décrits via plusieurs ressources et création d'une unique ressource pour chaque.

In [ ]:
# Détection des communes et arrondissements dupliqués et création d'une source centrale (?newLandmark) selon le code INSEE.
def clean_ban_graph(graphdb_url, repository_name, factoids_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    query1 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; rdfs:label {label_var}.
            BIND({norm_label_function} AS {norm_label_var})
        }}
    }}
    """

    query2 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?insee {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; geofla:numInsee ?insee.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; geofla:numInsee ?insee.
        }}
    }}
    """

    # Détection des doublons sur les codes postaux

    query3 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?postalCode {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
        }}
    }}
    """

    # Détection des doublons sur les voies (même nom et appartient à la même commune ou au même arrondissement)

    query4 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?label ?district WHERE {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; skos:hiddenLabel ?label.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; skos:hiddenLabel ?label.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
        }}
    }}
    """
    # Transfert des données des ressources temporaires vers les permanentes.

    query5 = prefixes + f"""
    DELETE {{
        GRAPH ?g {{
            ?s ?p ?tmpLandmark.
            ?tmpLandmark ?p ?o.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?s ?p ?landmark.
            ?landmark ?p ?o.
        }}
    }}
    WHERE {{
        ?landmark skos:exactMatch ?tmpLandmark.
        GRAPH ?g {{
            {{?tmpLandmark ?p ?o}} UNION {{?s ?p ?tmpLandmark}}
          }}
    }} ; 

    DELETE {{
        ?landmark skos:exactMatch ?tmpLandmark.
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    """

    queries = [query1, query2, query3, query4, query5]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

### Ajout d'événéments / de changements ainsi que des attributs à tous les repères

In [ ]:
def add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_graph_uri):
    """
    Ajouter des éléments comme les changements, les événements, les attributs et leurs versions
    """

    query = f"""
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    DELETE {{
        GRAPH ?g {{ 
            ?landmark geo:asWKT ?geom; geofla:numInsee ?inseeCode.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?landmark addr:hasAttribute ?nameAttribute, ?geomAttribute.
            ?nameAttribute a addr:Attribute; addr:isAttributeType atype:NameAttribute; addr:version ?versionNameAttribute1, ?versionNameAttribute2.
            ?geomAttribute a addr:Attribute; addr:isAttributeType atype:GeometryAttribute; addr:version ?versionGeomAttribute.
            ?versionNameAttribute1 a addr:AttributeVersion; addr:versionValue ?label.
            ?versionNameAttribute2 a addr:AttributeVersion; addr:versionValue ?inseeCode.
            ?versionGeomAttribute a addr:AttributeVersion; addr:versionValue ?geom.
        }}  
    }}
    WHERE {{
        {{
            SELECT * {{
                BIND({factoids_graph_uri.n3()} AS ?g)
                GRAPH ?g {{
                    ?landmark a addr:Landmark; rdfs:label ?label.
                    OPTIONAL {{?landmark geo:asWKT ?geom}}
                    OPTIONAL {{?landmark geofla:numInsee ?inseeCode}}
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_AN_", STRUUID())) AS ?nameAttribute)
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_ANV_", STRUUID())) AS ?versionNameAttribute1)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(ban:)), "BAN_ANV_", STRUUID())), ?x) AS ?versionNameAttribute2)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "BAN_AG_", STRUUID())), ?x) AS ?geomAttribute)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "BAN_AGV_", STRUUID())), ?x) AS ?versionGeomAttribute)
    }}
    """
    
    gd.update_query(query, graphdb_url, repository_name)


#### Définition d'un processus de création des données de la BAN

In [ ]:
def create_factoid_process_ban(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, facts_graph_name,
                               ban_csv_file, ban_mapping_file, ban_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la BAN
    """

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))

    # A partir des fichiers csv décrivant les adresses de la BAN dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, ban_csv_file, ban_mapping_file, ban_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)

    # Nettoyer les données en fusionnant les doublons après l'import dans GraphDB
    clean_ban_graph(graphdb_url, repository_name, factoids_graph_uri)

    # Ajout d'éléments manquants
    add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)

    # L'URI ci-dessous définit la source liée à la BAN
    ban_factoids_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_BAN")
    create_factoids_ban(graphdb_url, repository_name, ban_factoids_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_factoids_resources_links(graphdb_url, repository_name, ban_factoids_uri, factoids_graph_uri)

### Voies de Paris via Wikidata

#### Création des données liées à la source

In [ ]:
def create_factoids_wikidata(graphdb_url, repository_name, factoids_refactoids_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {factoids_refactoids_uri.n3()} a rico:Record;
                    rdfs:label "Wikidata"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Sélection des voies de Paris sur Wikidata via une requête de sélection

Via une requête SPARQL de sélection, on récupère une liste de ressources à créér :
* pour chaque ressource décrivant une voie dans Paris, on créé autant de ressources qu'elle a de nom officiel. La place de la Nation ([wd:Q1573359](https://www.wikidata.org/entity/Q1573359)) a quatre noms officiels donc on aura 4 ressources. Pour celles qui n'ont pas de nom officiel, on créé une seule ressource dont le nom est le label en français
* ces ressources créées seront liées à celle de Wikidata dont elle provient via `skos:closeMatch`
* chaque ressource créée suit la structure définie par l'ontologie
* le résultat de la requête est stocké dans un fichier csv qui sera converti en graphe de connaissance via Ontotext Refine et un fichier de mapping 

In [ ]:
def get_paris_thoroughfares_from_wikidata(out_csv_file):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX source: <http://rdf.geohistoricaldata.org/id/address/sources/wikidata/> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
    PREFIX wb: <http://wikiba.se/ontology#>
    PREFIX time: <http://www.w3.org/2006/time#>

    SELECT ?landmarkId ?voie ?nomOff ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal
    WHERE {{

        BIND(CONCAT("LM_", STRUUID()) AS ?landmarkId)
        {{
            SELECT DISTINCT * WHERE {{
                {{?voie p:P361 [ps:P361 wd:Q16024163].}}UNION{{?voie p:P361 [ps:P361 wd:Q107311481].}}
                {{?voie p:P1448 ?nomOffSt.
                    ?nomOffSt ps:P1448 ?nomOff.
                    OPTIONAL {{?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}}
                    OPTIONAL {{?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}}
                }}UNION{{
                    ?voie rdfs:label ?nomOff.
                    FILTER (LANG(?nomOff) = "fr")
                    MINUS {{?voie p:P1448 ?nomOffSt}}
                }}
            }}
        }}
        
        BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
                        IF(?dateStartPrecRaw = 10, time:unitMonth,
                            IF(?dateStartPrecRaw = 9, time:unitYear,
                                IF(?dateStartPrecRaw = 8, time:unitDecade,
                                    IF(?dateStartPrecRaw = 7, time:unitCentury,
                                        IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                                    )))))) AS ?dateStartPrec)
        BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
                        IF(?dateEndPrecRaw = 10, time:unitMonth,
                            IF(?dateEndPrecRaw = 9, time:unitYear,
                                IF(?dateEndPrecRaw = 8, time:unitDecade,
                                    IF(?dateEndPrecRaw = 7, time:unitCentury,
                                        IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                                    )))))) AS ?dateEndPrec)
    }}
    """

    wd.save_select_query_as_csv_file(query, out_csv_file)

#### Définition d'un processus de création des données de Wikidata

In [ ]:
def create_factoid_process_wikidata(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, facts_graph_name,
                               wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de Wikidata
    """

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))

    # Récupération des données via le endpoint de Wikidatadans un fichier CSV
    # get_paris_thoroughfares_from_wikidata(wdpt_csv_file)

    # A partir des fichiers csv décrivant les adresses de la BAN dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)

    # L'URI ci-dessous définit la source liée à Wikidata
    wdpt_factoids_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_WD")
    create_factoids_wikidata(graphdb_url, repository_name, wdpt_factoids_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_factoids_resources_links(graphdb_url, repository_name, wdpt_factoids_uri, factoids_graph_uri)

# Création des faits

## Création de données dans le graphe des faits

### Définition de fonctions pour peupler le graphe des faits

In [ ]:
def create_unlinked_resources(graphdb_url, repository_name, refactoids_class:str, refactoids_prefix:str, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    # Créer des ressources en tant que fait et créer un lien de provenance
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?resource owl:sameAs ?sourceResource.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?resource a ?type.
        }}
    }}
    WHERE {{
        ?type rdfs:subClassOf* addr:{refactoids_class}.
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceResource a ?type.
        }}
        MINUS {{?sourceResource owl:sameAs [a addr:{refactoids_class}]}}
        BIND(URI(CONCAT(STR(URI(facts:)), "{refactoids_prefix}_", STRUUID())) AS ?resource)
    }}
    """
    gd.update_query(query, graphdb_url, repository_name)

def create_linked_between_similar_thoroughfares(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour les repères de type VOIE définis dans le graphe nommé `factoids_graph_uri`, les lier avec un repère de même type défini dans `facts_graph_uri` s'ils ont un nom similaire.
    Le lien créé est mis dans `factoids_facts_graph_uri`.
    """
    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?factsLandmark owl:sameAs ?sourceLandmark.
        }}
    }}
    WHERE {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?factsLandmark a addr:Landmark.
           }}
        MINUS {{?factsLandmark owl:sameAs ?sourceLandmark}}
        ?sourceLandmark skos:hiddenLabel ?label.
        ?factsLandmark skos:hiddenLabel ?label.
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def store_interesting_implicit_triples(graphdb_url, repository_name, tmp_graph_uri:URIRef):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    
    INSERT {{
        GRAPH {tmp_graph_uri.n3()} {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        ?s ?p ?o.

        MINUS {{
            GRAPH ?g {{ ?s ?p ?o. }}
        }}
        FILTER(?p in (addr:isAttributeType,addr:isChangeType,addr:isLandmarkType,addr:isLandmarkRelationType,
            addr:hasTime,addr:timeCalendar,addr:timePrecision,addr:timeStamp,
            addr:hasAttribute,addr:version,addr:versionValue,addr:appliedTo,addr:dependsOn,addr:makesEffective,addr:outdates,
            addr:targets,addr:locatum,addr:relatum,addr:firstStep,addr:nextStep,
            rdfs:label,skos:hiddenLabel,skos:closeMatch,rico:isOrWasDescribedBy
        ))
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def get_facts_implicit_triples(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef, tmp_graph_uri:URIRef):
    """
    Tous les triplets intéressants (selon la propriété du triplet) ont été stockés dans un graphe nommé temporaire.
    Transférer les triplets dont :
    - les sujets sont des ressources RS définies dans le graphe des faits (il existe <RS a ?rtype> dans le graphe des faits) 
    - les objets ne sont pas des ressources RO définies dans le graphe des factoïdes (celles tel qu'il n'existe pas <RO a ?rtype> dans le graphe des factoïdes)
    """
    
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    
    DELETE {{
        GRAPH ?gt {{
            ?s ?p ?o.
        }}
    }}
    INSERT {{
        GRAPH ?gf {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        BIND ({facts_graph_uri.n3()} AS ?gf)
        BIND ({factoids_graph_uri.n3()} AS ?gs)
        BIND ({tmp_graph_uri.n3()} AS ?gt)

        GRAPH ?gt {{
            ?s ?p ?o.
        }}

        GRAPH ?gf {{
            ?s a ?sType.
        }}

        OPTIONAL {{
            GRAPH ?g {{?o a ?oType}}
        }}

        BIND(isIRI(?o) AS ?isIRI)
        BIND(IF(BOUND(?g) && ?g != ?gs, "true"^^xsd:boolean, "false"^^xsd:boolean) AS ?iriInFacts)
        FILTER(?isIRI = "false"^^xsd:boolean || ?iriInFacts = "true"^^xsd:boolean)
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def transfer_implicit_triples(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    tmp_graph_name = "tmp_named_graph"
    tmp_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, tmp_graph_name))

    # On stocke dans un graphe nommé temporaire les triplets implicites qui sont intéressants (selon la propriété du triplet)
    store_interesting_implicit_triples(graphdb_url, repository_name, tmp_graph_uri)

    # Suppression des liens owl:sameAs pour casser les liens implicites qui ont été stockés explicitement dans le graphe nommé
    remove_all_same_as_triples(graphdb_url, repository_name)

    # Refaire les inférences pour supprimer notamment les liens owl:sameAs implicites qui ne sont pas supprimés
    gd.reinfer_repository(graphdb_url, repository_name)

    # Transférer les triplets stockés dans le graphe nommé temporaire qui ne sont pas liés aux factoïdes
    get_facts_implicit_triples(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri, tmp_graph_uri)
    
    # Supprimer tous les triples du graphe nommé temporaire
    gd.remove_graph(graphdb_url, repository_name, tmp_graph_name)


def remove_all_same_as_triples(graphdb_url, repository_name):
    query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    DELETE {?s owl:sameAs ?o} WHERE {?s owl:sameAs ?o}
    """
    
    gd.update_query(query, graphdb_url, repository_name)

### Création de voies à partir des différentes sources

La création de voies se fait de la manière suivante :
* on normalise les labels des repères, qui sont stockés dans un graphe nommé temporaire
* on sélectionne une source
* pour chaque ressource définie dans la source, on regarde si elle existe dans le graphe des faits (via des critères pré-définis comme le nom)
    * si la ressource existe, on crée un lien `?a addr:isReferencedBy ?b` décrivant le fait que la ressource existe
    * sinon, on la crée et on ajoute un lien `?a addr:isCreatedBy ?b`
* on refait le processus suivant pour une autre source jusqu'à les avoir toutes faites

In [ ]:
def links_factoids_with_facts(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    create_linked_between_similar_thoroughfares(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    resource_classes = {"LM": "Landmark", "ADDR": "Address", "ATTR": "Attribute", "AV":"AttributeVersion", "CG": "Change", "EV":"Event", "TE": "TemporalEntity", "LR": "LandmarkRelation"}
    for prefix, class_name in resource_classes.items():
        create_unlinked_resources(graphdb_url, repository_name, class_name, prefix, factoids_graph_uri, facts_graph_uri)

### Réinitialisation du répertoire

La réinitialisation du répertoire :
* export du graphe des faits dans un fichier TTL
* suppression de tous les triplets du répertoire (explicites comme implicites)
* réimport des fichiers TTL de l'ontologie et du graphe des faits

:TODO: voir si c'est possible de supprimer facilement les triplets implicites pour éviter de faire appel à cette fonction.

In [ ]:
def reload_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_uri, ont_file, ontology_named_graph_name):    
    # Exporter le graphe des faits dans un fichier TTL
    gd.export_data_from_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_uri)

    # Réinitialiser le répertoire et le remplir une nouvelle fois avec l'ontologie et le graphe des faits
    gd.clear_repository(graphdb_url, repository_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, ont_file, ontology_named_graph_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, facts_ttl_file, graph_uri=facts_graph_uri)

### Import des factoïdes dans le graphe des faits

L'import des factoïdes dans le graphe des faits se fait en trois étapes :
* liaison des éléments du graphe des factoides avec celui des faits (`links_factoids_with_facts()`)
* avec l'étape précédente, on a des inférences qui se font et `transfer_implicit_triples()` récupère les triplets implicites intéressants pour les expliciter en les mettant dans le graphe des faits
* `reload_repository()` exporte le graphe des faits dans un fichier temporaire afin de nettoyer le répertoire (suppression de triplets implicites inutiles), le graphe des faits est rechargé dans le répertoire.

In [ ]:
def import_factoids_in_facts(graphdb_url, repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file):
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))

    links_factoids_with_facts(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    transfer_implicit_triples(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    reload_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_uri, ont_file, ontology_named_graph_name)

## Processus final et itératif

Pour chaque source, on fait les choses suivantes :
* création du graphe des factoïdes associés à la source (`create_factoid_process_XXXX()`)
    * on part d'un fichier décrivant les données brutes (souvent c'est un fichier tabulaire)
    * on utilise Ontotext-Refine pour le convertir en un graphe de connaissances selon l'ontologie définie. La manière de convertir est donnée par un fichier de mapping et le graphe est stocké dans un fichier TTL 
    * le fichier TTL est importé dans GraphDB dans un graphe nommé des factoïdes
* voir la section précédente pour comprendre ce que fait `import_factoids_in_facts()`

In [ ]:
# Import des données de la ville de Paris
create_factoid_process_ville_paris(graphdb_url, repository_name, ontorefine_url, ontorefine_cmd, factoids_graph_name, facts_graph_name,
                                   vpta_csv_file, vptc_csv_file, vpta_mapping_file, vptc_mapping_file, vpta_kg_file, vptc_kg_file)
import_factoids_in_facts(graphdb_url, repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)

In [ ]:
# Import des données de la BAN
create_factoid_process_ban(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, facts_graph_name, bpa_csv_file, bpa_mapping_file, bpa_kg_file)
import_factoids_in_facts(graphdb_url, repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)

In [ ]:
# Import des données de Wikidata
create_factoid_process_wikidata(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, facts_graph_name, wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file)
import_factoids_in_facts(graphdb_url, repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)